# Business calendars.

It's saturday 2022/09/04 at 20:30 local time. You need to order something. Is the business open? 

In [1]:
from datetime import datetime,date,time,timedelta

In [2]:
class BusinessCalendar(object):
    def __init__(self, open_days=[0,1,2,3,4,5,6], open_from=time(8), open_to=time(18), holidays=None, valid_from=date(1970,1,1), valid_to=date(2060,1,1), ) -> None:
        """
        open_days: integer monday=0 to sunday=6
        open_from: datetime.time 
        open_to: datetime.time
        holidays: list of dates
        valid_from: start datetime of calendar
        valid_to: start datetime of calendar
        """
        if not isinstance(open_days, list) and all(0<=i<=6 for i in open_days):
            raise TypeError("Expects open_days as a list of integers from 0 to 6.")
        self.open_days = open_days

        
        if not isinstance(valid_from, (date,datetime)):
            raise TypeError
        self.valid_from = valid_from

        if not isinstance(valid_to, (date,datetime)):
            raise TypeError
        self.valid_to = valid_to

        if not isinstance(open_from, time):
            raise TypeError
        self.open_from = open_from

        if not isinstance(open_to, time):
            raise TypeError
        self.open_to = open_to

        if not isinstance(holidays, list) and all(isinstance(i,date) for i in holidays):
            raise TypeError

        if holidays is None:
            holidays = []
        self.holidays = holidays  # consider using https://pypi.org/project/holidays/

    def __str__(self) -> str:
        return f"Calendar({self.open_days}, {self.open_from}-{self.open_to} excl. {self.holidays} within {self.valid_from}-{self.valid_to}"

    def is_open(self, now):
        if not isinstance(now, datetime):
            raise TypeError
        return all([
           now.weekday() in self.open_days,
           now.date() not in self.holidays,
           self.open_from <= now.time() < self.open_to,
           self.valid_from <= now.date() <= self.valid_to
        ])
        
    def open_next(self, now):
        if self.is_open(now):
            return now
        if not (self.valid_from <= now.date() < self.valid_to):
            return None  # outside calendar limit.
        
        new = datetime(now.year, now.month, now.day, self.open_from.hour, self.open_from.minute)
        while True:
            if new.date() in self.holidays or new.weekday() not in self.open_days:
                new += timedelta(days=1)
            else:
                break
        if self.valid_from <= new.date() < self.valid_to:
            return new
        return None

Let's check it.

In [3]:
calendar = BusinessCalendar(open_days=[0,1,2,3,4,5], open_from=time(10), open_to=time(20,30), holidays=[date(2022,9,24)])


In [4]:
calendar.is_open(now=datetime(2022,9,24,20,0)) == False  # it's a holiday

True

In [5]:
calendar.open_next(now=datetime(2022,9,24,20,0)) == datetime(2022,9,26,10)  # 10am on Monday the 26th.

True

Ok. that works. Let's have multiple calendars: Off season and in season

In [6]:
class BusinessCalendars(object):
    def __init__(self, calendars=None) -> None:
        self.calendars = calendars if isinstance(calendars, list) and all(isinstance(i, BusinessCalendar) for i in calendars) else []
        self.calendars.sort(key=lambda x: x.valid_from)
    
    def __iadd__(self, other):
        if not isinstance(other, BusinessCalendar):
            raise TypeError
        self.calendars.append(other)
        self.calendars.sort(key=lambda x: x.valid_from)

    def is_open(self, now):
        return any(c.is_open(now) for c in self.calendars)
    
    def open_next(self, now):
        for c in self.calendars:
            n = c.open_next(now)
            if n is not None:
                return n

In [7]:
summer = BusinessCalendar(open_days=[0,1,2,3,4], open_from=time(9), open_to=time(21), holidays=[], valid_from=date(2022,4,1), valid_to=date(2022,9,1))
winter = BusinessCalendar(open_days=[0,1,2,3,4], open_from=time(10), open_to=time(22), holidays=[], valid_from=date(2022,9,1), valid_to=date(2023,4,1))
calendars = BusinessCalendars(calendars=[summer,winter])

start = datetime(2022,8,27)
for ts in range(20):
    now = start+timedelta(days=ts)
    new = calendars.open_next(now)
    print(now, "open:", calendars.is_open(now), "-->", new, "open:", calendars.is_open(new), "in", (new-now).days, "days")

2022-08-27 00:00:00 open: False --> 2022-08-29 09:00:00 open: True in 2 days
2022-08-28 00:00:00 open: False --> 2022-08-29 09:00:00 open: True in 1 days
2022-08-29 00:00:00 open: False --> 2022-08-29 09:00:00 open: True in 0 days
2022-08-30 00:00:00 open: False --> 2022-08-30 09:00:00 open: True in 0 days
2022-08-31 00:00:00 open: False --> 2022-08-31 09:00:00 open: True in 0 days
2022-09-01 00:00:00 open: False --> 2022-09-01 10:00:00 open: True in 0 days
2022-09-02 00:00:00 open: False --> 2022-09-02 10:00:00 open: True in 0 days
2022-09-03 00:00:00 open: False --> 2022-09-05 10:00:00 open: True in 2 days
2022-09-04 00:00:00 open: False --> 2022-09-05 10:00:00 open: True in 1 days
2022-09-05 00:00:00 open: False --> 2022-09-05 10:00:00 open: True in 0 days
2022-09-06 00:00:00 open: False --> 2022-09-06 10:00:00 open: True in 0 days
2022-09-07 00:00:00 open: False --> 2022-09-07 10:00:00 open: True in 0 days
2022-09-08 00:00:00 open: False --> 2022-09-08 10:00:00 open: True in 0 days

Will 3 shifts of 7 hours work too?

|shift| start | end |
|---|---|---|
|1|0800|1500|
|2|1600|2300|
|3|0000|0700|

1 hour maintenance shutdown between shifts.

In [8]:
shift_1 = BusinessCalendar(open_days=[0,1,2,3,4], open_from=time(8), open_to=time(15), holidays=[], valid_from=date(2022,4,1), valid_to=date(2022,9,1))
shift_2 = BusinessCalendar(open_days=[0,1,2,3,4], open_from=time(16), open_to=time(23), holidays=[], valid_from=date(2022,4,1), valid_to=date(2022,9,1))
shift_3 = BusinessCalendar(open_days=[0,1,2,3,4], open_from=time(0), open_to=time(7), holidays=[], valid_from=date(2022,4,1), valid_to=date(2022,9,1))

calendar = BusinessCalendars([shift_1,shift_2,shift_3])

In [9]:
now = datetime(2022,4,21,6,10)
for h in range(24):
    now += timedelta(hours=1)
    print(now, "is open:", calendar.is_open(now))

2022-04-21 07:10:00 is open: False
2022-04-21 08:10:00 is open: True
2022-04-21 09:10:00 is open: True
2022-04-21 10:10:00 is open: True
2022-04-21 11:10:00 is open: True
2022-04-21 12:10:00 is open: True
2022-04-21 13:10:00 is open: True
2022-04-21 14:10:00 is open: True
2022-04-21 15:10:00 is open: False
2022-04-21 16:10:00 is open: True
2022-04-21 17:10:00 is open: True
2022-04-21 18:10:00 is open: True
2022-04-21 19:10:00 is open: True
2022-04-21 20:10:00 is open: True
2022-04-21 21:10:00 is open: True
2022-04-21 22:10:00 is open: True
2022-04-21 23:10:00 is open: False
2022-04-22 00:10:00 is open: True
2022-04-22 01:10:00 is open: True
2022-04-22 02:10:00 is open: True
2022-04-22 03:10:00 is open: True
2022-04-22 04:10:00 is open: True
2022-04-22 05:10:00 is open: True
2022-04-22 06:10:00 is open: True
